In [42]:
using Complementarity
using Suppressor

In [32]:
ld0 = .6 #labor demand
ls0 = .4 #Leisure demand
le0 = ld0 + ls0 #Labor endowment
fixed_le0 = le0 #We modify le0 for the shock, so store it to not change important info

kd0 = .4 #captial demand
ke0 = kd0 #Captial endowment

sigma = .7 #Elasticity of substitution in production
theta = ld0/(ld0+kd0) #Labor value share
alpha = 1/(1+ls0) #Consumption value share

0.7142857142857143

In [39]:
function little_model()

    m = MCPModel()

    @variables(m,begin
        PL>=0, (start = 1,) #Wage rate
        PK>=0, (start = 1,) #Return to capital
        PY>=0, (start = 1,) #Output price
        Y>=0,  (start = 1,) #Production
        AY, (start = 1,)
        AL, (start = -ld0,)
        AK, (start = -kd0,)
    end)


    @NLexpression(m,C,(theta*PL^(1-sigma)+(1-theta)*PK^(1-sigma))^(1/(1-sigma)))



    @NLexpression(m,DY,alpha*(PL*le0+PK*ke0)/PY)
    @NLexpression(m,DL,(1-alpha)*(PL*le0+PK*ke0)/PL)
    @NLexpression(m,DK,0)

    @NLexpression(m,BY,0)
    @NLexpression(m,BL,le0)
    @NLexpression(m,BK,ke0)

    @mapping(m,def_AY,AY - 1)
    @mapping(m,def_AL,AL + ld0*(C/PL)^sigma)
    @mapping(m,def_AK,AK + kd0*(C/PK)^sigma)

    @mapping(m,profit, -PY*AY-PL*AL-PK*AK)

    @mapping(m,marketY,BY+AY*Y - DY)
    @mapping(m,marketK,BK+AK*Y - DK)
    @mapping(m,marketL,BL+AL*Y - DL)

    @complementarity(m,profit,Y)
    @complementarity(m,marketY,PY)
    @complementarity(m,marketK,PK)
    @complementarity(m,marketL,PL)
    @complementarity(m,def_AY,AY)
    @complementarity(m,def_AL,AL)
    @complementarity(m,def_AK,AK)



    return m
end


little_model (generic function with 1 method)

In [43]:
le0 = fixed_le0
benchmark = little_model()

path_output = @capture_out solveMCP(benchmark, cumulative_iteration_limit = 0)
print(generate_report(benchmark))

var_name	 value		 margin
Y		 1.0		 0.0
PY		 1.0		 0.0
PK		 1.0		 0.0
PL		 1.0		 0.0
AY		 1.0		 0.0
AL		 -0.6		 0.0
AK		 -0.4		 0.0


In [44]:
le0 = 1.1*fixed_le0

shock = little_model()

fix(shock[:PY],1,force=true)
path_output = @capture_out solveMCP(shock)

print(generate_report(shock))

var_name	 value		 margin
Y		 1.0545		 -0.0
PY		 1.0		 0.0
PK		 1.0787		 -0.0
PL		 0.9498		 -0.0
AY		 1.0		 0.0
AL		 -0.622		 0.0
AK		 -0.3793		 0.0
